In [ ]:
import feedparser
import dateutil.parser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import mongo_utils
from datetime import datetime as dt
import re


In [ ]:
dbname = mongo_utils.get_database()

In [ ]:
url='https://dariknews.bg/novini'

In [ ]:
manin_page = requests.get(url)
man_page_data = manin_page.text
main_page = BeautifulSoup(man_page_data, 'html.parser')
news_url=''    


In [ ]:
collection_name = dbname["dariknews_scrape_1"]
for scr_url in main_page.find_all('a', class_="gtm-CategoryArticlesList-click", href=True):
  try:
    nlink="https:" + scr_url['href']  
    if list(collection_name.find({"news_link" : nlink })):
            print("Article Exists")
    else:
        page = requests.get(nlink)
        #print(page)
        data = page.text
        #print(data)
        soup = BeautifulSoup(data, 'html.parser')
        text_body= soup.find("div", class_="article-text-inner-wrapper io-article-body").get_text()
        text=text_body.replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
        print(text)
        news_title=soup.find('title').get_text()
        date_time=soup.find('time', class_="time-stamp", datetime=True)
        date_posted=dt.strptime(date_time['datetime'], '%Y-%m-%d %H:%M:%S').timestamp()
        news_payload = {
        "date_posted" : date_posted,
        "news_link" : nlink,
        "news_text" : text,
        "news_title" : news_title
        }
        print(news_payload)
        collection_name = dbname["dariknews_scrape_1"]
        collection_name.insert_many([news_payload])
  except:
    pass